In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-06-21 12:27:00,143 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Tweets 50 / 300 /skip gram

In [2]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model')
model.init_sims(replace=True)

2017-06-21 12:27:00,619 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model
2017-06-21 12:27:06,872 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.wv.* with mmap=None
2017-06-21 12:27:06,872 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.wv.syn0.npy with mmap=None
2017-06-21 12:27:10,819 : INFO : setting ignored attribute syn0norm to None
2017-06-21 12:27:10,821 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model.syn1neg.npy with mmap=None
2017-06-21 12:27:27,006 : INFO : setting ignored attribute cum_table to None
2017-06-21 12:27:27,030 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-300-skip.model
2017-06-21 12:27:30,509 : INFO : precomputing L2-norms of word weight vectors


In [3]:
print(model.most_similar(positive=['rss', 'haha']))

[(u'rs', 0.8790924549102783), (u'kk', 0.8604964017868042), (u'hahaha', 0.8581379652023315), (u'hahah', 0.8580098748207092), (u'hehe', 0.8567013740539551), (u'rsrs', 0.8455395698547363), (u'hehehe', 0.8401827812194824), (u'kkk', 0.8391278386116028), (u'hahahaha', 0.8145782947540283), (u'kkkk', 0.8133679032325745)]


In [4]:
print(model.most_similar(positive=[':D', ':/']))

[(u':(', 0.8271794319152832), (u';)', 0.8041931390762329), (u':)', 0.8021442890167236), (u':))', 0.7625139951705933), (u':c', 0.7616134881973267), (u'*-*', 0.7583409547805786), (u'=)', 0.7568512558937073), (u':3', 0.7542413473129272), (u':P', 0.7475375533103943), (u'^^', 0.744448184967041)]


In [5]:
print(model.most_similar(positive=['fofo', 'cute']))

[(u'fofinho', 0.7537943124771118), (u'lindinho', 0.7081056237220764), (u'meigo', 0.7032421827316284), (u'fofooo', 0.6875720620155334), (u'fofoooo', 0.6844723224639893), (u'fofo!', 0.6831097602844238), (u'cute!', 0.6810519695281982), (u'mordivel', 0.6774948835372925), (u'mord\xedvel', 0.6755068302154541), (u'fofo,', 0.6729605197906494)]


In [6]:
num_features= 300

In [7]:
df = pd.read_csv('/home/bahbbc/personality-normalized-word2vec.csv', encoding='utf-8')
df.shape

(1039, 186)

In [8]:
df.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [9]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [10]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['extraversion_m']
    evaluate_prediction(predictions, target)

In [11]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [12]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [13]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [14]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-06-21 12:28:19,621 : WARNING : cannot compute similarity with no input []
2017-06-21 12:28:19,961 : WARNING : cannot compute similarity with no input []
2017-06-21 12:28:19,967 : WARNING : cannot compute similarity with no input []
2017-06-21 12:28:20,614 : WARNING : cannot compute similarity with no input []
2017-06-21 12:28:20,871 : WARNING : cannot compute similarity with no input []
2017-06-21 12:28:21,235 : WARNING : cannot compute similarity with no input []
2017-06-21 12:28:21,944 : WARNING : cannot compute similarity with no input []
2017-06-21 12:28:22,038 : WARNING : cannot compute similarity with no input []
2017-06-21 12:28:22,268 : WARNING : cannot compute similarity with no input []
2017-06-21 12:28:22,437 : WARNING : cannot compute similarity with no input []
2017-06-21 12:28:22,497 : WARNING : cannot compute similarity with no input []


CPU times: user 3.9 s, sys: 62.2 ms, total: 3.96 s
Wall time: 3.99 s


In [15]:
del model

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [17]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   24.4s finished


CPU times: user 4.83 s, sys: 93 ms, total: 4.92 s
Wall time: 29.3 s


In [18]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['extraversion_m']

In [19]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,100,42
1,79,91


In [20]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.56      0.70      0.62       142
          1       0.68      0.54      0.60       170

avg / total       0.63      0.61      0.61       312



In [21]:
accuracy_score(yt, yp)

0.61217948717948723

In [22]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [23]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['extraversion_m']

In [24]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       387
          1       1.00      1.00      1.00       340

avg / total       1.00      1.00      1.00       727



In [25]:
print accuracy_score(ytt, ytp)

0.998624484182
